In [ ]:
#LSTM
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Set random seed for reproducibility
np.random.seed(42)

# Load the data
data = pd.read_csv('IBM.csv')

# Normalize the data
scaler = MinMaxScaler(feature_range=(-1, 1))
normalized_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1)).flatten() # Assuming 'Close' is the column you want to predict

# Data Preprocessing Function modified for multi-step
def create_sequences(data, sequence_length, steps):
    xs, ys = [], []
    for i in range(len(data) - sequence_length - steps + 1):
        x = data[i:(i + sequence_length)]
        y = data[i + sequence_length:i + sequence_length + steps]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Create sequences with modified function
sequence_length = 1  # Updated sequence length for your use case
steps = 1  # Number of future steps to predict
X, y = create_sequences(normalized_data, sequence_length, steps)

# Split the data
split = int(len(X) * 0.9)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Reshape for LSTM [samples, time steps, features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Build the LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(sequence_length, 1)))
model.add(Dropout(0.2))
model.add(Dense(steps))  # The output layer should have 'steps' units if predicting multiple steps

# Compile the model
model.compile(optimizer=Adam(0.001), loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1, validation_data=(X_test, y_test))

# Predicting
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Evaluate the model
mae_train = mean_absolute_error(y_train, train_predict)
rmse_train = np.sqrt(mean_squared_error(y_train, train_predict))

mae_test = mean_absolute_error(y_test, test_predict)
rmse_test = np.sqrt(mean_squared_error(y_test, test_predict))

print(f"Training Data - MAE: {mae_train}, RMSE: {rmse_train}")
print(f"Testing Data - MAE: {mae_test}, RMSE: {rmse_test}")

# If you want to plot the predictions (optional)
import matplotlib.pyplot as plt
plt.plot(y_test)
plt.plot(test_predict)
plt.show()


In [ ]:
#LSTM-CNN 
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.optimizers import Adam

# Set random seed for reproducibility
np.random.seed(42)

# Load the data from a CSV file
data_path = 'IBM.csv'  # Update this to the correct path where IBM.csv is located
data = pd.read_csv(data_path)

# Assuming 'Close' is the column you want to predict, adjust if needed
data = data['Close'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(-1, 1))
normalized_data = scaler.fit_transform(data).flatten()

# Data Preprocessing Function for multi-step
def create_sequences(data, sequence_length, steps):
    xs, ys = [], []
    for i in range(len(data) - sequence_length - steps + 1):
        x = data[i:(i + sequence_length)]
        y = data[i + sequence_length:i + sequence_length + steps]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Create sequences with modified function
sequence_length = 7  # This is the window size of your sequence
steps = 1  # This is the prediction horizon
X, y = create_sequences(normalized_data, sequence_length, steps)

# Split the data
split = int(len(X) * 0.9)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Reshape for LSTM-CNN [samples, time steps, features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Define the LSTM-CNN model
model = Sequential()

# LSTM layer with return_sequences=True for the following CNN layers
model.add(LSTM(64, input_shape=(sequence_length, 1), return_sequences=True))
model.add(Dropout(0.2))

# CNN layers
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())

# Final dense layers
model.add(Dense(50, activation='relu'))
model.add(Dense(steps))  # Adjust 'steps' if you're predicting more than one future step

# Compile the model
model.compile(optimizer=Adam(0.001), loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1, validation_data=(X_test, y_test))

# Predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Evaluate the model using MAE and RMSE
mae_train = mean_absolute_error(y_train, train_predict)
rmse_train = np.sqrt(mean_squared_error(y_train, train_predict))

mae_test = mean_absolute_error(y_test, test_predict)
rmse_test = np.sqrt(mean_squared_error(y_test, test_predict))

print(f"Training Data - MAE: {mae_train}, RMSE: {rmse_train}")
print(f"Testing Data - MAE: {mae_test}, RMSE: {rmse_test}")


In [ ]:
#SVR
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from torch.optim.lr_scheduler import OneCycleLR
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.models import Sequential

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Load the data
data_path = 'lorenz.txt'  # Update this to the correct path where lorenz.txt is located
data = np.loadtxt(data_path)

# Normalize the data
scaler = MinMaxScaler(feature_range=(-1, 1))
normalized_data = scaler.fit_transform(data.reshape(-1, 1)).flatten()

# Data Preprocessing Function modified for multi-step
def create_sequences(data, sequence_length, steps):
    xs, ys = [], []
    for i in range(len(data) - sequence_length - steps + 1):
        x = data[i:(i + sequence_length)]
        y = data[i + sequence_length:i + sequence_length + steps]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Create sequences with modified function
sequence_length = 1  
steps = 1  # Number of future steps to predict
X, y = create_sequences(normalized_data, sequence_length, steps)

# Split the data
split = int(len(X) * 0.9)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Instantiate and fit the SVR model
# Note: You may want to tune these hyperparameters
svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
svr_rbf.fit(X_train, y_train.ravel())

# Predictions
train_predict = svr_rbf.predict(X_train)
test_predict = svr_rbf.predict(X_test)

# Evaluate the model using MAE and RMSE on the scaled data
mae_train = mean_absolute_error(y_train, train_predict)
rmse_train = np.sqrt(mean_squared_error(y_train, train_predict))

mae_test = mean_absolute_error(y_test, test_predict)
rmse_test = np.sqrt(mean_squared_error(y_test, test_predict))

print(f"Training Data - MAE: {mae_train}, RMSE: {rmse_train}")
print(f"Testing Data - MAE: {mae_test}, RMSE: {rmse_test}")
